In [1]:
import datasets
from datasets import load_dataset

In [2]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# NLTK Imports
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [3]:
from string import punctuation
from heapq import nlargest

In [4]:
dataset = datasets.load_dataset('cnn_dailymail', '3.0.0')

Using the latest cached version of the module from C:\Users\Namrata\.cache\huggingface\modules\datasets_modules\datasets\cnn_dailymail\1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de (last modified on Sun Jun 11 11:02:01 2023) since it couldn't be found locally at cnn_dailymail., or remotely on the Hugging Face Hub.
Found cached dataset cnn_dailymail (C:/Users/Namrata/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
train_dataset = dataset["train"]

In [6]:
# Print the first few examples from the train split
for example in train_dataset[:5]:
    print(example)

article
highlights
id


In [7]:
train_dataset.column_names

['article', 'highlights', 'id']

In [8]:
train_dataset['article'][1]

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to foll

In [9]:
train_dataset['highlights'][1]

'Mentally ill inmates in Miami are housed on the "forgotten floor"\nJudge Steven Leifman says most are there as a result of "avoidable felonies"\nWhile CNN tours facility, patient shouts: "I am the son of the president"\nLeifman says the system is unjust and he\'s fighting for change .'

# Rouge Score

In [10]:
from rouge import Rouge

def calculate_rouge_score(summary, reference):
    rouge = Rouge()
    scores = rouge.get_scores(summary, reference)
    return scores

## Spacy Summarize

In [11]:
def spacy_summarize(text_content, percent):
    # Frequency Based Summarization using Spacy.
    # Build a List of Stopwords

    stop_words = list(STOP_WORDS)

    # import punctuations from strings library.
    punctuation_items = punctuation + '\n'

    # Loading en_core_web_sm in Spacy
    nlp = spacy.load('en_core_web_sm')

    # Build an NLP Object
    nlp_object = nlp(text_content)

    # Create the dictionary with key as words and value as number of times word is repeated.
    # Scoring words by its occurrence.
    word_frequencies = {}
    for word in nlp_object:
        if word.text.lower() not in stop_words:
            if word.text.lower() not in punctuation_items:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1

    # Finding frequency of most occurring word
    max_frequency = max(word_frequencies.values())

    # Divide Number of occurrences of all words by the max_frequency
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word] / max_frequency

    # Save a sentence-tokenized copy of text
    sentence_token = [sentence for sentence in nlp_object.sents]

    # Create the dictionary with key as sentences and value as sum of each important word.
    # Scoring sentences by its words.
    sentence_scores = {}
    for sent in sentence_token:
        sentence = sent.text.split(" ")
        for word in sentence:
            if word.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word.lower()]
                else:
                    sentence_scores[sent] += word_frequencies[word.lower()]

    
    # Finding number of sentences and applying percentage on it: since we require to show most X% lines in summary.
    select_length = int(len(sentence_token) * (int(percent) / 100))

    # Using nlargest library to get the top x% weighted sentences.
    summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)

    # Later joining it to get the final summarized text.
    final_summary = [word.text for word in summary]
    summary = ' '.join(final_summary)

    
#     score = calculate_rouge_score(summary, sum2)

    # Returning NLTK Summarization Output
    return summary


In [12]:
generated_summaries_for_spacy = []

In [13]:
for article in train_dataset['article'][:5]:
    summary = spacy_summarize(article, 30)
    generated_summaries_for_spacy.append(summary)

In [14]:
# generated_summaries_for_spacy

In [16]:
for summ, ref in zip(generated_summaries_for_spacy, train_dataset['highlights'][:5]):
    rouge_score = calculate_rouge_score(summ, ref)
    print(rouge_score)

[{'rouge-1': {'r': 0.6111111111111112, 'p': 0.13414634146341464, 'f': 0.21999999704800005}, 'rouge-2': {'r': 0.4864864864864865, 'p': 0.08144796380090498, 'f': 0.13953488126404665}, 'rouge-l': {'r': 0.5833333333333334, 'p': 0.12804878048780488, 'f': 0.20999999704800004}}]
[{'rouge-1': {'r': 0.5121951219512195, 'p': 0.11602209944751381, 'f': 0.18918918617766417}, 'rouge-2': {'r': 0.2608695652173913, 'p': 0.04363636363636364, 'f': 0.0747663526848537}, 'rouge-l': {'r': 0.4634146341463415, 'p': 0.10497237569060773, 'f': 0.17117116815964617}}]
[{'rouge-1': {'r': 0.43243243243243246, 'p': 0.08247422680412371, 'f': 0.13852813583778417}, 'rouge-2': {'r': 0.1282051282051282, 'p': 0.015723270440251572, 'f': 0.028011202535602613}, 'rouge-l': {'r': 0.43243243243243246, 'p': 0.08247422680412371, 'f': 0.13852813583778417}}]
[{'rouge-1': {'r': 0.5416666666666666, 'p': 0.13, 'f': 0.20967741623309058}, 'rouge-2': {'r': 0.043478260869565216, 'p': 0.0078125, 'f': 0.013245030530240405}, 'rouge-l': {'r': 0

# Lsa Summarization

In [19]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [20]:
def sumy_lsa_summarize(text_content, percent):
    # Latent Semantic Analysis is a unsupervised learning algorithm that can be used for extractive text summarization.
    # Initializing the parser

    parser = PlaintextParser.from_string(text_content, Tokenizer("english"))
    # Initialize the stemmer
    stemmer = Stemmer('english')
    # Initializing the summarizer
    summarizer = LsaSummarizer(stemmer)
    summarizer.stop_words = get_stop_words('english')

    # Finding number of sentences and applying percentage on it: since sumy requires number of lines
    sentence_token = sent_tokenize(text_content)
    select_length = int(len(sentence_token) * (int(percent) / 100))

    # Evaluating and saving the Summary
    summary = ""
    for sentence in summarizer(parser.document, sentences_count=select_length):
        summary += str(sentence)

    
    # Returning NLTK Summarization Output
    return summary

In [21]:
generated_summaries_for_lsa = []
for article in train_dataset['article'][:5]:
    summary = sumy_lsa_summarize(article, 30)
    generated_summaries_for_lsa.append(summary)

In [22]:
for summ, ref in zip(generated_summaries_for_lsa, train_dataset['highlights'][:5]):
    rouge_score = calculate_rouge_score(summ, ref)
    print(rouge_score)

[{'rouge-1': {'r': 0.5, 'p': 0.11764705882352941, 'f': 0.19047618739229027}, 'rouge-2': {'r': 0.32432432432432434, 'p': 0.05714285714285714, 'f': 0.09716598935566889}, 'rouge-l': {'r': 0.4444444444444444, 'p': 0.10457516339869281, 'f': 0.16931216622826914}}]
[{'rouge-1': {'r': 0.4634146341463415, 'p': 0.09405940594059406, 'f': 0.15637859801791737}, 'rouge-2': {'r': 0.10869565217391304, 'p': 0.01718213058419244, 'f': 0.029673588147117798}, 'rouge-l': {'r': 0.3902439024390244, 'p': 0.07920792079207921, 'f': 0.13168723999322604}}]
[{'rouge-1': {'r': 0.35135135135135137, 'p': 0.06220095693779904, 'f': 0.10569105435488142}, 'rouge-2': {'r': 0.07692307692307693, 'p': 0.009287925696594427, 'f': 0.01657458371279898}, 'rouge-l': {'r': 0.35135135135135137, 'p': 0.06220095693779904, 'f': 0.10569105435488142}}]
[{'rouge-1': {'r': 0.5, 'p': 0.1111111111111111, 'f': 0.18181817884297521}, 'rouge-2': {'r': 0.043478260869565216, 'p': 0.006802721088435374, 'f': 0.011764703542561018}, 'rouge-l': {'r': 0.

# nltk summarize

In [23]:
def nltk_summarize(text_content, percent):
    # Frequency Based Summarization using NLTK
    # Store a tokenized copy of text, using NLTK's recommended word tokenizer
    tokens = word_tokenize(text_content)

    # Import the stop words from NLTK toolkit
    stop_words = stopwords.words('english')

    # import punctuations from strings library.
    punctuation_items = punctuation + '\n'

    # Create the dictionary with key as words and value as number of times word is repeated.
    # Scoring words by its occurrence.
    word_frequencies = {}
    for word in tokens:
        if word.lower() not in stop_words:
            if word.lower() not in punctuation_items:
                if word not in word_frequencies.keys():
                    word_frequencies[word] = 1
                else:
                    word_frequencies[word] += 1

    # Finding frequency of most occurring word
    max_frequency = max(word_frequencies.values())

    # Divide Number of occurrences of all words by the max_frequency
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word] / max_frequency

    # Save a sentence-tokenized copy of text
    sentence_token = sent_tokenize(text_content)

    # Create the dictionary with key as sentences and value as sum of each important word.
    # Scoring sentences by its words.
    sentence_scores = {}
    for sent in sentence_token:
        sentence = sent.split(" ")
        for word in sentence:
            if word.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word.lower()]
                else:
                    sentence_scores[sent] += word_frequencies[word.lower()]

    # Finding number of sentences and applying percentage on it: since we require to show most X% lines in summary.
    select_length = int(len(sentence_token) * (int(percent) / 100))

    # Using nlargest library to get the top x% weighted sentences.
    summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)

    # Later joining it to get the final summarized text.
    final_summary = [word for word in summary]
    summary = ' '.join(final_summary)


    # Returning NLTK Summarization Output
    return summary


In [25]:
generated_summaries_for_nltk = []
for article in train_dataset['article'][:5]:
    summary = nltk_summarize(article, 30)
    generated_summaries_for_nltk.append(summary)

In [27]:
for summ, ref in zip(generated_summaries_for_nltk, train_dataset['highlights'][:5]):
    rouge_score = calculate_rouge_score(summ, ref)
    print(rouge_score)

[{'rouge-1': {'r': 0.6111111111111112, 'p': 0.13836477987421383, 'f': 0.22564102263037478}, 'rouge-2': {'r': 0.4864864864864865, 'p': 0.08144796380090498, 'f': 0.13953488126404665}, 'rouge-l': {'r': 0.5833333333333334, 'p': 0.1320754716981132, 'f': 0.21538461237396456}}]
[{'rouge-1': {'r': 0.5365853658536586, 'p': 0.11518324607329843, 'f': 0.1896551695039388}, 'rouge-2': {'r': 0.2826086956521739, 'p': 0.04421768707482993, 'f': 0.0764705858955018}, 'rouge-l': {'r': 0.4634146341463415, 'p': 0.09947643979057591, 'f': 0.16379310053842155}}]
[{'rouge-1': {'r': 0.43243243243243246, 'p': 0.08791208791208792, 'f': 0.14611871865307233}, 'rouge-2': {'r': 0.10256410256410256, 'p': 0.012944983818770227, 'f': 0.02298850375693636}, 'rouge-l': {'r': 0.43243243243243246, 'p': 0.08791208791208792, 'f': 0.14611871865307233}}]
[{'rouge-1': {'r': 0.5, 'p': 0.12244897959183673, 'f': 0.19672130831496912}, 'rouge-2': {'r': 0.043478260869565216, 'p': 0.008064516129032258, 'f': 0.01360543953723037}, 'rouge-l':

# gensim

In [31]:
from gensim.summarization.summarizer import summarize

In [32]:
def gensim_summarize(text_content, percent):
    # TextRank Summarization using Gensim Library.
    # Split is false, gensim return strings joined by "\n". if true, gensim will return list
    summary = summarize(text_content, ratio=(int(percent) / 100), split=False).replace("\n", " ")
    
    # Returning NLTK Summarization Output
    return summary

In [33]:
generated_summaries_for_gensim = []
for article in train_dataset['article'][:5]:
    summary = gensim_summarize(article, 30)
    generated_summaries_for_gensim.append(summary)

In [35]:
for summ, ref in zip(generated_summaries_for_gensim, train_dataset['highlights'][:5]):
    rouge_score = calculate_rouge_score(summ, ref)
    print(rouge_score)

[{'rouge-1': {'r': 0.8888888888888888, 'p': 0.18604651162790697, 'f': 0.3076923048298817}, 'rouge-2': {'r': 0.7567567567567568, 'p': 0.11914893617021277, 'f': 0.2058823505906683}, 'rouge-l': {'r': 0.8888888888888888, 'p': 0.18604651162790697, 'f': 0.3076923048298817}}]
[{'rouge-1': {'r': 0.4878048780487805, 'p': 0.12269938650306748, 'f': 0.19607842816080356}, 'rouge-2': {'r': 0.1956521739130435, 'p': 0.037815126050420166, 'f': 0.06338027897540183}, 'rouge-l': {'r': 0.4878048780487805, 'p': 0.12269938650306748, 'f': 0.19607842816080356}}]
[{'rouge-1': {'r': 0.4864864864864865, 'p': 0.1016949152542373, 'f': 0.16822429620534549}, 'rouge-2': {'r': 0.15384615384615385, 'p': 0.02142857142857143, 'f': 0.037617552712728965}, 'rouge-l': {'r': 0.4594594594594595, 'p': 0.096045197740113, 'f': 0.15887850181282212}}]
[{'rouge-1': {'r': 0.5, 'p': 0.15, 'f': 0.23076922721893492}, 'rouge-2': {'r': 0.043478260869565216, 'p': 0.008849557522123894, 'f': 0.014705879542604341}, 'rouge-l': {'r': 0.5, 'p': 0

# Luhn summarization algorithm

In [36]:
def sumy_luhn_summarize(text_content, percent):

    # A naive approach based on TF-IDF and looking at the “window size” of non-important words between words of high
    # importance. It also assigns higher weights to sentences occurring near the beginning of a document.
    # Initializing the parser
    parser = PlaintextParser.from_string(text_content, Tokenizer("english"))
    # Initialize the stemmer
    stemmer = Stemmer('english')
    # Initializing the summarizer
    summarizer = LuhnSummarizer(stemmer)
    summarizer.stop_words = get_stop_words('english')

    # Finding number of sentences and applying percentage on it: since sumy requires number of lines
    sentence_token = sent_tokenize(text_content)
    select_length = int(len(sentence_token) * (int(percent) / 100))

    # Evaluating and saving the Summary
    summary = ""
    for sentence in summarizer(parser.document, sentences_count=select_length):
        summary += str(sentence)

    # Returning NLTK Summarization Output
    return summary


In [37]:
generated_summaries_for_luhn = []
for article in train_dataset['article'][:5]:
    summary = sumy_luhn_summarize(article, 30)
    generated_summaries_for_luhn.append(summary)

In [38]:
for summ, ref in zip(generated_summaries_for_luhn, train_dataset['highlights'][:5]):
    rouge_score = calculate_rouge_score(summ, ref)
    print(rouge_score)

[{'rouge-1': {'r': 0.8888888888888888, 'p': 0.22377622377622378, 'f': 0.357541896227958}, 'rouge-2': {'r': 0.7567567567567568, 'p': 0.14507772020725387, 'f': 0.24347825816975427}, 'rouge-l': {'r': 0.8888888888888888, 'p': 0.22377622377622378, 'f': 0.357541896227958}}]
[{'rouge-1': {'r': 0.5121951219512195, 'p': 0.12, 'f': 0.19444444136874145}, 'rouge-2': {'r': 0.1956521739130435, 'p': 0.033962264150943396, 'f': 0.05787781098417108}, 'rouge-l': {'r': 0.4878048780487805, 'p': 0.11428571428571428, 'f': 0.1851851821094822}}]
[{'rouge-1': {'r': 0.4594594594594595, 'p': 0.10365853658536585, 'f': 0.16915422585183537}, 'rouge-2': {'r': 0.15384615384615385, 'p': 0.02158273381294964, 'f': 0.037854887432057366}, 'rouge-l': {'r': 0.4594594594594595, 'p': 0.10365853658536585, 'f': 0.16915422585183537}}]
[{'rouge-1': {'r': 0.5, 'p': 0.1188118811881188, 'f': 0.19199999689728003}, 'rouge-2': {'r': 0.043478260869565216, 'p': 0.007518796992481203, 'f': 0.01282051030654224}, 'rouge-l': {'r': 0.5, 'p': 0.